# Data Science Lab Project 2022 Winter Session (TF-IDF part)

In [7]:
!wget "https://dbdmg.polito.it/dbdmg_web/wp-content/uploads/2021/12/DSL2122_january_dataset.zip" 
!unzip 'DSL2122_january_dataset.zip' -d '/content/data'

--2022-01-20 10:57:25--  https://dbdmg.polito.it/dbdmg_web/wp-content/uploads/2021/12/DSL2122_january_dataset.zip
Risoluzione di dbdmg.polito.it (dbdmg.polito.it)... 130.192.163.163
Connessione a dbdmg.polito.it (dbdmg.polito.it)|130.192.163.163|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 18640208 (18M) [application/zip]
Salvataggio in: «DSL2122_january_dataset.zip.1»

DSL2122_january_dat 100%[===================>]  17,78M  6,36MB/s    in 2,8s    

2022-01-20 10:57:28 (6,36 MB/s) - «DSL2122_january_dataset.zip.1» salvato [18640208/18640208]

Archive:  DSL2122_january_dataset.zip
checkdir:  cannot create extraction directory: /content/data
           No such file or directory


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import wordnet
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import string as st
import re

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/giovannimantegna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
pip install emot --upgrade

Note: you may need to restart the kernel to use updated packages.


In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/giovannimantegna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
data = pd.read_csv("development.csv", parse_dates=['date'])

/Users/giovannimantegna/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [13]:
data_ev = pd.read_csv("evaluation.csv", parse_dates=['date'])

In [14]:
ids = data["ids"]
data_clean = data.drop(data[ids.isin(ids[ids.duplicated()])].index)
data_clean = data_clean.drop_duplicates(subset=['text'])

In [15]:
import emot 
emot_obj = emot.core.emot() 

# preprocessing

In [16]:
def load_dict_smileys():
  return {
  ";)":"smirk",
  ":-)": "smiley",
  ":)": "smiley",
  ":d": "smiley",
  "xd": "smiley",
  ":')": "smiley",
  ":'d": "smiley",
  ":3": "smiley",
  ":]": "smiley",
  ":^)": "smiley",
  ":-]": "smiley",
  ":-3": "smiley",
  ":->": "smiley",
  ":))": "smiley",
  "8-)": "cool",
  "8)": "cool",
  "8-d": "cool",
  "b)": "cool",
  "bd": "cool",
  ":-}": "smiley",
  ":>": "smiley",
  ":}": "smiley",
  ":o)": "smiley",
  ":c)": "smiley",
  "=)": "smiley",
  "=d": "smiley",
  "=]": "smiley",
  ":-d": "smiley",
  ":o": "surprised",
  "=o": "surprised",
  ":0": "surprised",
  ":(": "sad",
  ":c": "sad",
  "=(": "sad",
  "=c": "sad",
  ":-(": "sad",
  ":-c": "sad",
  ":-<": "sad",
  ":<": "sad",
  ":[": "sad",
  ":[[": "sad",
  ":{": "sad",
  ":'(": "crying",
  ":P": "playful",
  "xp": "playful",
  "=P": "playful",
  "xb": "playful",
  "<3": "love",
  "</3": "sad",
  ":/": "worried",
  ":-/": "worried"
  }

In [17]:
from functools import partial

def removeUnicode(text):
    """ Removes unicode strings like "\u002c" and "x96" """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
    text = re.sub(r'[^\x00-\x7f]',r'',text)
    return text

def removeURL(text):
    """ Removes url """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
    text = re.sub(r'#([^\s]+)', '', text)
    return text


def removeNumbers(text):
    """ Removes integers """
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

def replaceMultiExclamationMark(text):
    """ Replaces repetitions of exlamation marks """
    text = re.sub(r"(\!)\1+", ' multiExclamation ', text) 
    return text

def replaceMultiQuestionMark(text):
    """ Replaces repetitions of question marks """
    text = re.sub(r"(\?)\1+", ' multiQuestion ', text) 
    return text

def replaceMultiStopMark(text):
    """ Replaces repetitions of stop marks """
    text = re.sub(r"(\.)\1+", ' multiStop ', text) 
    return text


""" Creates a dictionary with slangs and their equivalents and replaces them """
with open('slang.txt') as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2])
    for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True) # longest first for regex
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)].replace(" ", "_"))


""" Replaces contractions from a string to their equivalents """
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
def replaceContraction(text):
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word



def replace(word, pos=None):
    """ Creates a set of all antonyms for the word and if there is only one antonym, it returns it """
    antonyms = set()
    for syn in wordnet.synsets(word, pos=pos):
      for lemma in syn.lemmas():
        for antonym in lemma.antonyms():
          antonyms.add(antonym.name())
    if len(antonyms) == 1:
      return antonyms.pop()
    else:
      return None

def replaceNegations(text):
    """ Finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym """
    list_text = text.split()
    i, l = 0, len(list_text)
    words = []
    while i < l:
      word = list_text[i]
      if word == 'not' and i+1 < l:
        ant = replace(list_text[i+1])
        if ant:
          text = text.replace(list_text[i+1], ant)
          words.append(ant)
          i += 2
          continue
      i += 1
    if words:
      text = text.replace("not", "")
    return text


def addNotTag(text):
    """ Finds "not,never,no" and adds the tag NEG_ to all words that follow until the next punctuation """
    transformed = re.sub(r'\b(?:not|never|no)\b[\w\s][\w]+', 
        lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), 
        text,
        flags=re.IGNORECASE)
    return transformed


def replaceElongatedWord(text):

  tokens = text.split() 
  for w in tokens:
    final_word = replaceElongated(w)
    if len(final_word)>1:
        text = text.replace(w, final_word)
  return text

def detect_emoticons_dict(tweet):
  tweet = re.sub("(http|ftp|https)://[\w-]+(\.[\w-]+)+([\w.,@?^=%&amp;:/~+#-]*[\w@?^=%&amp;/~+#-])?","", tweet)
  tweet = re.sub(r'(\W)(?=\1)', '', tweet)
  tweet = re.sub('[a-z0-9]+:', '', tweet)
  tweet = tweet.lower()
  smilies = load_dict_smileys()
  for key,val in smilies.items():
    if key in tweet or key[::-1] in tweet:
      new_tweet = tweet.replace(key, val)
      tweet = new_tweet
  return tweet

def detect_emoticons_library(text):
  text = re.sub("(http|ftp|https)://[\w-]+(\.[\w-]+)+([\w.,@?^=%&amp;:/~+#-]*[\w@?^=%&amp;/~+#-])?","", text)
  text = re.sub(r'(\W)(?=\1)', '', text) 
  text = re.sub('[a-z0-9]+:', '', text)
  emot = emot_obj.emoticons(str(text).lower().strip())
  if emot['flag']:
    for count, face in enumerate(emot['value']):
      text = text.lower().replace(face, emot['mean'][count].split()[0].lower())
  return text

In [18]:
def preprocessing(text):
  text = replaceMultiExclamationMark(text)
  text = replaceMultiQuestionMark(text)
  text = replaceMultiStopMark(text)
  text = text.replace("&quot|&amp|&lt|&gt|&nbsp|&apos|&cent|&pound|&yen|&euro|&copy|&reg", "")
  text = re.sub("(a*ha+h[ha]*)","ahah ",text)
  text = re.sub("(o?l+o+l+[ol]*)","lol ",text) 
  text = removeUnicode(text) 
  text = removeURL(text)
  text = detect_emoticons_library(text)
  text = detect_emoticons_dict(text)
  text = replaceElongatedWord(text)
  text = replaceSlang(text)
  text = replaceContraction(text)
  text = removeNumbers(text)
  text = replaceNegations(text)
  text = "".join([char if char not in st.punctuation.replace("_","") else " " for char in text])
  text = addNotTag(text)
  text = re.sub("[^A-Za-z0–9_. ]+","",text)
  return text

In [19]:
data_clean["text_cleaned"] = data_clean["text"].apply(lambda x:  preprocessing(x))

In [20]:
data_ev["text_cleaned"] = data_ev["text"].apply(lambda x:  preprocessing(x))

In [21]:
data_clean = data_clean.drop_duplicates(subset=['text_cleaned'])

# Using TF-IDF model for most similar sentence

In [22]:
tv = feature_extraction.text.TfidfVectorizer(max_features=70000, ngram_range=(1,2))

In [23]:
tv_df = tv.fit_transform(data_clean["text_cleaned"][:20000].values)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tv_df, tv_df)

In [25]:
indices = pd.Series(range(20000), index=data_clean["text_cleaned"][:20000])

def get_similarity(title, cosine_sim, indices):
    # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwsie similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]
    # Get the movie indices
    text_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    return data_clean[['text_cleaned', 'sentiment']].iloc[text_indices]

In [26]:
get_similarity("bored and its raining ", cosine_sim, indices)

,text_cleaned,sentiment
793,urgh its raining bad,0
5488,is kind_of bored and extremely tired,0
2499,bye tweeters multistop its already pm here a...,1
4880,its raining again what happened to mr sun,0
1959,i am bored and tired i can go home,0
17207,bored again,0
5614,man i wanted to go outside today but now its r...,0
15048,raining raining raining i love rain,1
16349,its raining multistop best thing to do is stay...,1
10989,its official its raining in essex no NEG_bq to...,0


# Model Performance Evaluation Functions

In [27]:
####Evaluation metrics


from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.preprocessing import label_binarize
from scipy import interp
from sklearn.metrics import roc_curve, auc 


def get_metrics(true_labels, predicted_labels):
    
    print('Accuracy:', np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        4))
    print('Precision:', np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('Recall:', np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
    print('F1 Score:', np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted'),
                        4))
                        

def train_predict_model(classifier, 
                        train_features, train_labels, 
                        test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    return predictions    


def display_confusion_matrix(true_labels, predicted_labels, classes=[1,0]):
    
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm = metrics.confusion_matrix(y_true=true_labels, y_pred=predicted_labels, 
                                  labels=classes)
    cm_frame = pd.DataFrame(data=cm, 
                            columns=pd.MultiIndex(levels=[['Predicted:'], classes], 
                                                  codes=level_labels), 
                            index=pd.MultiIndex(levels=[['Actual:'], classes], 
                                                codes=level_labels)) 
    print(cm_frame) 
    
def display_classification_report(true_labels, predicted_labels, classes=[1,0]):

    report = metrics.classification_report(y_true=true_labels, 
                                           y_pred=predicted_labels, 
                                           labels=classes) 
    print(report)
    
    
    
def display_model_performance_metrics(true_labels, predicted_labels, classes=[1,0]):
    print('Model Performance metrics:')
    print('-'*30)
    get_metrics(true_labels=true_labels, predicted_labels=predicted_labels)
    print('\nModel Classification report:')
    print('-'*30)
    display_classification_report(true_labels=true_labels, predicted_labels=predicted_labels, 
                                  classes=classes)
    print('\nPrediction Confusion Matrix:')
    print('-'*30)
    display_confusion_matrix(true_labels=true_labels, predicted_labels=predicted_labels, 
                             classes=classes)

# Model Selection


In [28]:
tv_train_features = tv.fit_transform(data_clean["text_cleaned"].values)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(tv_train_features, data_clean["sentiment"].values, test_size=0.20, random_state=42)

**LinearSVC**

In [30]:
from sklearn.svm import LinearSVC

lsvm = LinearSVC(random_state= 42)

lsvm.fit(X_train, y_train)
y_pred = lsvm.predict(X_test)

In [31]:
display_model_performance_metrics(true_labels=y_test, predicted_labels=y_pred, classes=[0,1])

Model Performance metrics:
------------------------------
Accuracy: 0.7953
Precision: 0.7944
Recall: 0.7953
F1 Score: 0.7944

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.77      0.73      0.75     18696
           1       0.81      0.84      0.83     25762

    accuracy                           0.80     44458
   macro avg       0.79      0.79      0.79     44458
weighted avg       0.79      0.80      0.79     44458


Prediction Confusion Matrix:
------------------------------
          Predicted:       
                   0      1
Actual: 0      13658   5038
        1       4064  21698


In [32]:
f1_score(y_test, y_pred, average='macro')

0.7883453069616426

**RandomForestClassifier**

In [33]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1, n_estimators= 500, random_state=42)

In [34]:
y_pred = rfc.fit(X_train, y_train).predict(X_test)

In [35]:
display_model_performance_metrics(true_labels=y_test, predicted_labels=y_pred, classes=[0,1])

Model Performance metrics:
------------------------------
Accuracy: 0.7817
Precision: 0.7807
Recall: 0.7817
F1 Score: 0.7797

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.77      0.69      0.73     18696
           1       0.79      0.85      0.82     25762

    accuracy                           0.78     44458
   macro avg       0.78      0.77      0.77     44458
weighted avg       0.78      0.78      0.78     44458


Prediction Confusion Matrix:
------------------------------
          Predicted:       
                   0      1
Actual: 0      12871   5825
        1       3881  21881


In [36]:
f1_score(y_test, y_pred, average='macro')

0.7723306686331299

**LogisticRegression**

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
lr = LogisticRegression(max_iter=500, random_state=42)

In [39]:
y_pred = lr.fit(X_train, y_train).predict(X_test)

In [40]:
display_model_performance_metrics(true_labels=y_test, predicted_labels=y_pred, classes=[0,1])

Model Performance metrics:
------------------------------
Accuracy: 0.8056
Precision: 0.8049
Recall: 0.8056
F1 Score: 0.8041

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           0       0.79      0.73      0.76     18696
           1       0.81      0.86      0.84     25762

    accuracy                           0.81     44458
   macro avg       0.80      0.79      0.80     44458
weighted avg       0.80      0.81      0.80     44458


Prediction Confusion Matrix:
------------------------------
          Predicted:       
                   0      1
Actual: 0      13557   5139
        1       3505  22257


In [41]:
f1_score(y_test, y_pred, average='macro')

0.7978271530069108

# Gridsearch

In [42]:
X_train, X_test, y_train, y_test = train_test_split(data_clean["text_cleaned"].values, data_clean["sentiment"].values, test_size=0.20, random_state=42)

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression


tfidf = TfidfVectorizer(max_features=70000, ngram_range=(1,2))

porter = PorterStemmer()
liemmatizer = WordNetLemmatizer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

def tokenizer_lem(text):
    return [liemmatizer.lemmatize(word) for word in text.split()]


stop = stopwords.words('english')

**Grid for LogisticRegression**

In [ ]:
param_grid = { 'vect__ngram_range': [(1, 2), (1, 3)],
               'vect__tokenizer': [tokenizer_porter, tokenizer_lem, None],
               'vect__stop_words': [stop, None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}
              

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, max_iter=500))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='f1',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train,y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


In [ ]:
print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best f1: %.3f' % gs_lr_tfidf.best_score_)

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print('f1 in test: %.3f' % clf.score(X_test, y_test))

# Model without date and user

In [ ]:
tv_train_features = tv.fit_transform(data_clean["text_cleaned"].values)

In [ ]:
tv_features_ev = tv.transform(data_ev["text_cleaned"].values)

In [ ]:
lr = LogisticRegression(penalty='l2', max_iter=500, C=1,  solver = 'newton-cg')

In [ ]:
lr.fit(tv_train_features, data_clean["sentiment"].values)

In [ ]:
y_pred_lr = lr.predict(tv_features_ev)

In [ ]:
import csv

header = 'Id Predicted'
header = header.split()

file = open('pred.csv', 'w', newline='')

with file:
    writer = csv.writer(file)
    writer.writerow(header)

for i in range(len(y_pred_lr)):
  pred = y_pred_lr[i]
  to_append = f'{i} {pred}'
  file = open('pred.csv', 'a', newline='')
  with file:
      writer = csv.writer(file)
      writer.writerow(to_append.split())

# Improvement using date and user

In [ ]:
import datetime as dt
data_clean['date'] = pd.to_datetime(data_clean['date'])
data_clean['date']= data_clean['date'].map(dt.datetime.toordinal)

data_ev['date'] = pd.to_datetime(data_ev['date'])
data_ev['date']= data_ev['date'].map(dt.datetime.toordinal)

## Training-test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_clean, data_clean["sentiment"].values, test_size=0.20, random_state=42)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown = 'ignore')

In [ ]:
user = enc.fit_transform(X_train['user'].values.reshape(-1,1))

In [ ]:
user_ev = enc.transform(X_test['user'].values.reshape(-1,1) )

In [ ]:
tv_train_features = tv.fit_transform(X_train["text_cleaned"].values)

In [ ]:
tv_train_features_ev = tv.transform(X_test["text_cleaned"].values)

In [ ]:
import scipy.sparse
X = scipy.sparse.hstack([tv_train_features, user])

In [ ]:
X_ev = scipy.sparse.hstack([tv_train_features_ev, user_ev])

In [ ]:
lr.fit(X, y_train)

In [ ]:
y_pred_lr = lr.predict(X)

In [ ]:
y_pred_lr_ev = lr.predict(X_ev)

In [ ]:
X_test["pred_lr"] = y_pred_lr_ev

In [ ]:
X_train["pred_lr"] = y_pred_lr

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1, n_estimators= 500, random_state = 42)

In [ ]:
rfc.fit(X_train[['pred_lr', 'date']].values, y_train )

In [ ]:
y_pred_rf_ev = rfc.predict(X_test[['pred_lr', 'date']].values)

In [ ]:
display_model_performance_metrics(true_labels=y_test, predicted_labels=y_pred_rf_ev, classes=[0,1])

## Evaluation

In [ ]:
tv = feature_extraction.text.TfidfVectorizer(max_features=70000, ngram_range=(1,2))

In [ ]:
tv_train_features = tv.fit_transform(data_clean["text_cleaned"].values)

In [ ]:
tv_train_features_ev = tv.transform(data_ev["text_cleaned"].values)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown = 'ignore')

In [ ]:
user = enc.fit_transform(data_clean['user'].values.reshape(-1,1))

In [ ]:
user_ev = enc.transform(data_ev['user'].values.reshape(-1,1) )

In [ ]:
import scipy.sparse
X = scipy.sparse.hstack([tv_train_features, user])

In [ ]:
X_ev = scipy.sparse.hstack([tv_train_features_ev, user_ev])

In [ ]:
lr.fit(X, data_clean["sentiment"].values)

In [ ]:
y_pred_lr = lr.predict(X)

In [ ]:
y_pred_lr_ev = lr.predict(X_ev)

In [ ]:
data_ev["pred_lr"] = y_pred_lr_ev

In [ ]:
data_clean["pred_lr"] = y_pred_lr

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1, n_estimators= 500, random_state=42)

In [ ]:
rfc.fit(data_clean[['pred_lr', 'date']].values,  data_clean["sentiment"].values)

In [ ]:
y_pred_rf_ev = rfc.predict(data_ev[['pred_lr', 'date']].values)

In [ ]:
import csv

header = 'Id Predicted'
header = header.split()

file = open('pred_cascade.csv', 'w', newline='')

with file:
    writer = csv.writer(file)
    writer.writerow(header)

for i in range(len(y_pred_rf_ev)):
  pred = y_pred_rf_ev[i]
  to_append = f'{i} {pred}'
  file = open('pred_cascade.csv', 'a', newline='')
  with file:
      writer = csv.writer(file)
      writer.writerow(to_append.split())

# Evaluation no cascade approach with date and user

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown = 'ignore')

In [ ]:
tv = feature_extraction.text.TfidfVectorizer(max_features=70000, ngram_range=(1,2))

In [ ]:
tv_train_features = tv.fit_transform(data_clean["text_cleaned"].values)

In [ ]:
tv_train_features_ev = tv.transform(data_ev["text_cleaned"].values)

In [ ]:
user = enc.fit_transform(data_clean['user'].values.reshape(-1,1))

In [ ]:
user_ev = enc.transform(data_ev['user'].values.reshape(-1,1) )

In [ ]:
import scipy.sparse

In [ ]:
X = scipy.sparse.hstack([tv_train_features, user, data_clean['date'].values.reshape(-1,1)])

In [ ]:
X_ev = scipy.sparse.hstack([tv_train_features_ev, user_ev, data_ev['date'].values.reshape(-1,1)])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1, n_estimators= 100, random_state= 42)

In [ ]:
rfc.fit(X,  data_clean["sentiment"].values)

In [ ]:
y_pred_rf_ev = rfc.predict(X_ev)

In [ ]:
import csv

header = 'Id Predicted'
header = header.split()

file = open('pred_randomForest.csv', 'w', newline='')

with file:
    writer = csv.writer(file)
    writer.writerow(header)

for i in range(len(y_pred_rf_ev)):
  pred = y_pred_rf_ev[i]
  to_append = f'{i} {pred}'
  file = open('pred_randomForest.csv', 'a', newline='')
  with file:
      writer = csv.writer(file)
      writer.writerow(to_append.split())

In [ ]:
from imblearn.metrics import classification_report_imbalanced

In [ ]:
print(classification_report_imbalanced(y_test, y_pred))